In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
import jieba
import numpy as np
from sklearn.impute import SimpleImputer

 特征提取练习：文本特征抽取

In [10]:
def couvec():
    # min_df：每个单词在文本中出现的次数
    vec = CountVectorizer() # min_df=2:'is', 'life', 'python', 'short']
    # fit_transform():转换数据  默认单字母的字符串不输出(类似i，a)
    # 三个样本
    res = vec.fit_transform([
        'Life is short, u need python life',
        'life is not short, u not dislike python',
        'life is too long'])
    # 打印分离出来的每一个值:符合出现次数的字符
    print(vec.get_feature_names())
    print('-'*50)
    # 输出样本字符的稀疏矩阵(词频)
    print(res)
    print(type(res))
    print(res.toarray())
    # 把样本的特征和矩阵里的词频对应
    print(vec.inverse_transform(res))

couvec()

['dislike', 'is', 'life', 'long', 'need', 'not', 'python', 'short', 'too']
--------------------------------------------------
  (0, 2)	2
  (0, 1)	1
  (0, 7)	1
  (0, 4)	1
  (0, 6)	1
  (1, 2)	1
  (1, 1)	1
  (1, 7)	1
  (1, 6)	1
  (1, 5)	2
  (1, 0)	1
  (2, 2)	1
  (2, 1)	1
  (2, 8)	1
  (2, 3)	1
<class 'scipy.sparse._csr.csr_matrix'>
[[0 1 2 0 1 0 1 1 0]
 [1 1 1 0 0 2 1 1 0]
 [0 1 1 1 0 0 0 0 1]]
[array(['life', 'is', 'short', 'need', 'python'], dtype='<U7'), array(['life', 'is', 'short', 'python', 'not', 'dislike'], dtype='<U7'), array(['life', 'is', 'too', 'long'], dtype='<U7')]


C:\Users\life\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
# 汉字文本数值化
def countvec():
    # 汉字特征值化，单个汉字字母都不统计，因为无意义
    ve = CountVectorizer()
    data = ve.fit_transform(['人生漫漫，我 在 孤岛上 迎接黎明， python python',
                             '人生 苦短，u need python'])
    print(ve.get_feature_names())
    print(data)
    print(data.toarray())
    print(ve.inverse_transform(data))

countvec()

['need', 'python', '人生', '人生漫漫', '孤岛上', '苦短', '迎接黎明']
  (0, 3)	1
  (0, 4)	1
  (0, 6)	1
  (0, 1)	2
  (1, 1)	1
  (1, 2)	1
  (1, 5)	1
  (1, 0)	1
[[0 2 0 1 1 0 1]
 [1 1 1 0 0 1 0]]
[array(['人生漫漫', '孤岛上', '迎接黎明', 'python'], dtype='<U6'), array(['python', '人生', '苦短', 'need'], dtype='<U6')]


C:\Users\life\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
# jieba中文分词
def cutword():
    word1 = jieba.cut('地也，你不分好歹何为地？天业，你错堪贤愚枉作天！')
    word2 = jieba.cut('纸上得来终觉浅，绝知此事要躬行。')
    word3 = jieba.cut('不畏浮云遮望眼，自缘身在最高层。')
    # 转换成列表
    print(type(word1))
    print('-'*50)
    # 切好放在列表上
    content1 = list(word1)
    content2 = list(word2)
    content3 = list(word3)
    # print(content1)
    # 列表打印成字符串
    cou1 = ' '.join(content1)
    cou2 = ' '.join(content2)
    cou3 = ' '.join(content3)
    return cou1,cou2,cou3

def hanzivec():
    '''
    分词后中文特征值化
    :return:
    '''
    cou1,cou2,cou3 = cutword()
    print(cou1,cou2,cou3)
    cv = CountVectorizer()
    print(cv.get_feature_names())
    data = cv.fit_transform([cou1,cou2,cou3])
    print(data.toarray())
    print(cv.inverse_transform(data))

hanzivec()

<class 'generator'>
--------------------------------------------------
地 也 ， 你 不分 好歹 何为 地 ？ 天业 ， 你 错堪 贤愚 枉 作天 ！ 纸上 得来 终觉 浅 ， 绝知 此事 要 躬行 。 不畏 浮云 遮 望眼 ， 自缘身 在 最高层 。


NotFittedError: Vocabulary not fitted or provided

TF-IDF TF:词频。idf:逆文档频率，lg(总文档数量/该词出现的文档数量)

In [13]:
def tfidfvec():
    # 评估文档重要程度

   cou1,cou2,cou3 = cutword()
   print(cou1,cou2,cou3)
   print(type([cou1,cou2,cou3]))
   tf = TfidfVectorizer()  # tf*idf
   data = tf.fit_transform([cou1,cou2,cou3])
   print('-'*50)
   print(tf.get_feature_names())
   print('-'*50)
   print(data.toarray())
   # 稀疏矩阵
   print(data)
   print('-'*50)
   print(tf.inverse_transform(data))
cutword()
tfidfvec()

<class 'generator'>
--------------------------------------------------
<class 'generator'>
--------------------------------------------------
地 也 ， 你 不分 好歹 何为 地 ？ 天业 ， 你 错堪 贤愚 枉 作天 ！ 纸上 得来 终觉 浅 ， 绝知 此事 要 躬行 。 不畏 浮云 遮 望眼 ， 自缘身 在 最高层 。
<class 'list'>
--------------------------------------------------
['不分', '不畏', '何为', '作天', '天业', '好歹', '得来', '最高层', '望眼', '此事', '浮云', '纸上', '终觉', '绝知', '自缘身', '贤愚', '躬行', '错堪']
--------------------------------------------------
[[0.37796447 0.         0.37796447 0.37796447 0.37796447 0.37796447
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.37796447 0.         0.37796447]
 [0.         0.         0.         0.         0.         0.
  0.40824829 0.         0.         0.40824829 0.         0.40824829
  0.40824829 0.40824829 0.         0.         0.40824829 0.        ]
 [0.         0.4472136  0.         0.         0.         0.
  0.         0.4472136  0.4472136  0.         0.4472136  0.
  0.         0.         

C:\Users\life\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


  特征处理

In [24]:
# 不同特征处理搞到同一个量纲上
'''
归一化处理:适用于传统精确数据场景
缺点：受极值影响
处理:x`=(x-min)/(max-min)
归一化更换区间处理（-1，1）：x``=x`*(mx-mi)+mi mx-1,mi--1'''
def mm():
    mm = MinMaxScaler(feature_range=(-1,1)) # 归一化范围（0，1）
    data = mm.fit_transform([[90, 2, 10, 40], [60, 4, 15, 45], [75, 3, 13, 46]])
    print(data)
    return None
mm()

[[ 1.         -1.         -1.         -1.        ]
 [-1.          1.          1.          0.66666667]
 [-0.99997     0.          0.2         1.        ]]


In [25]:
# 标准化处理不同特征值到同一量纲
def stand():
    '''
    通过对原始数据进行变化把数据变换到均值(期望)为0，标准差为1范围内
    机器学习找的是分布规律
    x`=(x-mean)/标准差
    :return:
    '''
    std = StandardScaler()
    data = std.fit_transform([[10000., -1., 3.], [2., 4., 2.], [4., 6., -1.]])
    # 有异常极值后
    print(data)
    print('-'*80)
    print(std.mean_)
    print('-'*80)
    print(std.var_)
    print('-'*80)
    print(std.n_samples_seen_)  # 样本数
stand()

[[ 1.41421354 -1.35873244  0.98058068]
 [-0.70731897  0.33968311  0.39223227]
 [-0.70689457  1.01904933 -1.37281295]]
--------------------------------------------------------------------------------
[3.33533333e+03 3.00000000e+00 1.33333333e+00]
--------------------------------------------------------------------------------
[2.22088916e+07 8.66666667e+00 2.88888889e+00]
--------------------------------------------------------------------------------
3


缺失值处理（1.填补:Simplelmputer(均值，中值，众值，固定值);2.删除）

In [26]:
def im():
    '''
    缺失值处理：填补
    :return:
    '''
    # NaN, nan,缺失值必须是这种形式，如果是？号(或者其他符号)，就要replace换成这种
    im = SimpleImputer(missing_values=np.nan, strategy='median') # 策略替换：均值，中值，众值，固定值
    data = im.fit_transform([[1, 2], [np.nan, 3], [7, 6], [3, 2]])
    print(data)
im()

[[1. 2.]
 [3. 3.]
 [7. 6.]
 [3. 2.]]


降维：特征数变少

可以提高模型训练速度

In [31]:
def var():
    var = VarianceThreshold(threshold=0)
    # 删除方差小于等于0的特征
    #默认只删除方差为0,threshold是方差阈值，删除比这个值小的那些特征
    data = var.fit_transform([[0, 2, 0, 3,2],
                              [0, 1, 4, 3,1],
                              [0, 1, 1, 3,2]])
    print(data)
    # 获得被选择特征值的列编号
    print('The surport is %s' % var.get_support(True))
var()

[[2 0 2]
 [1 4 1]
 [1 1 2]]
The surport is [1 2 4]
